# Introduction to Keras for engineers
https://keras.io/getting_started/intro_to_keras_for_engineers/

In [1]:
import keras
import numpy as np


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [7]:
np.dtype(x_train[0, 0, 0])

dtype('uint8')

In [8]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [9]:
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [10]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [11]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [12]:
x_train[0]

array([[[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
    

In [28]:
x_train = x_train[:10000, :, :, :]
y_train = y_train[:10000]

In [13]:
# Model parameters
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)


In [14]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)


In [31]:
batch_size = 128
epochs = 20

callbacks = [
    # keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    # keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    # keras.callbacks.TensorBoard(log_dir="C:/temp/logs", histogram_freq=1),
    keras.callbacks.RemoteMonitor(
        root="http://localhost:8080",
        path="/remotemonitor",
        field="data",
        headers=None,
        send_as_json=True,
    )

]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
score = model.evaluate(x_test, y_test, verbose=0)


Epoch 1/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 15s 219ms/step - acc: 0.9987 - loss: 0.0041 - val_acc: 0.9993 - val_loss: 0.0028
Epoch 2/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 14s 206ms/step - acc: 0.9987 - loss: 0.0040 - val_acc: 0.9987 - val_loss: 0.0055
Epoch 3/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 13s 190ms/step - acc: 0.9991 - loss: 0.0029 - val_acc: 0.9987 - val_loss: 0.0026
Epoch 4/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 13s 195ms/step - acc: 0.9993 - loss: 0.0025 - val_acc: 0.9987 - val_loss: 0.0022
Epoch 5/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 15s 223ms/step - acc: 0.9986 - loss: 0.0030 - val_acc: 1.0000 - val_loss: 5.7219e-04
Epoch 6/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 25s 370ms/step - acc: 0.9993 - loss: 0.0025 - val_acc: 0.9993 - val_loss: 0.0011
Epoch 7/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 19s 281ms/step - acc: 0.9985 - loss: 0.0039 - val_acc: 0.9987 - val_loss: 0.0026
Epoch 8/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 17s 257ms/step - acc: 0.9992 - loss: 0.0018 - val_acc: 0.9980 - val_loss: 0.0039
Epoch 9/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 27s 39

In [23]:
model.layers

[<Conv2D name=conv2d, built=True>,
 <Conv2D name=conv2d_1, built=True>,
 <MaxPooling2D name=max_pooling2d, built=True>,
 <Conv2D name=conv2d_2, built=True>,
 <Conv2D name=conv2d_3, built=True>,
 <GlobalAveragePooling2D name=global_average_pooling2d, built=True>,
 <Dropout name=dropout, built=True>,
 <Dense name=dense, built=True>]